In [ ]:
!pip install newspaper3k --quiet
!pip install justext --quiet

     |████████████████████████████████| 211 kB 4.0 MB/s 
     |████████████████████████████████| 93 kB 2.5 MB/s 
     |████████████████████████████████| 7.4 MB 20.4 MB/s 
     |████████████████████████████████| 81 kB 3.6 MB/s 
     |████████████████████████████████| 837 kB 4.1 MB/s 


In [ ]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import newspaper
from newspaper import Article
import pandas as pd
from tqdm.notebook import tqdm
import justext
from google.colab import files
from lxml.etree import ParseError

In [ ]:
file_path = '/content/business.csv'
table = pd.read_csv(file_path)

In [ ]:
table.columns

Index(['category', 'link'], dtype='object')

In [ ]:
feeds = []

In [ ]:
for item in table['link']:
  feeds.append(item)

In [ ]:
len(feeds)

256

In [ ]:
feeds_unique = list(set(feeds)) 
len(feeds_unique)

247

In [ ]:
feeds = feeds_unique
len(feeds)

247

In [ ]:
g_feeds = []
for feed in feeds:
    print(feed)
    try:
        response = requests.get(feed)
    except requests.exceptions.RequestException as e:
        print(e)
        r = "No response"
    if response.status_code == 200:
        g_feeds.append(feed)
    else:
        print(feed, response.status_code)
print(len(feeds))
print(len(g_feeds))

https://www.businesstravelnews.com/rss/business-travel-news
http://feeds.feedburner.com/daily-express-sport-news
http://feeds.bbci.co.uk/sport/0/cricket/rss.xml?edition=uk
http://www.npr.org/rss/rss.php?id=1004
http://www.npr.org/rss/rss.php?id=1004 500
http://rss.canada.com/get/?F7497
HTTPConnectionPool(host='rss.canada.com', port=80): Max retries exceeded with url: /get/?F7497 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f61f920c790>: Failed to establish a new connection: [Errno -2] Name or service not known'))
http://rss.canada.com/get/?F7497 500
http://www.oneindia.com/rss/news-india-fb.xml
http://www.oneindia.com/rss/news-india-fb.xml 403
http://rss.cnn.com/rss/edition_business.rss
http://www.thehindubusinessline.com/news/?service=rss
https://www.bworldonline.com/feed/
https://www.moveyourmoney.org.uk/feed/
http://www.latimes.com/nation/rss2.0.xml
http://www.latimes.com/nation/rss2.0.xml 403
http://rss.nytimes.com/services/xml/rss/nyt/Technology.x

In [ ]:
articles = []
title_l = []
text_l = []
for feed in g_feeds:
    try:
        response = requests.get(feed)
    except requests.exceptions.RequestException as e:
        print(e)
        r = "No response"
    webpage = response.content
    soup = BeautifulSoup(webpage, features='xml')
    items = soup.find_all('item')
    print("feed-->",feed)
    for item in items:
        link = item.find('link').text
        articles.append(link)
print(len(articles))


In [ ]:
# articles = []
# title_l = []
# text_l = []

In [ ]:
# file_path_links = '/content/news_links_final.csv'
# table_links = pd.read_csv(file_path_links)

In [ ]:
# articles_df = table_links['URL']
# articles = articles_df.values.tolist()

In [ ]:
# articles[155]

'http://www.rttnews.com/2283198/uncertain-mood-prevails-on-wall-street.aspx\\?type=glcom'

In [ ]:
# len(articles)

221860

In [ ]:
def get_content(content):
    # print("Into get_content")
    try:
      raw = requests.get(content,timeout=10)
      if raw.status_code == 200 and len(raw.content)>0:
        # print("raw.content-->",)
        paragraphs = justext.justext(raw.content, justext.get_stoplist("English"))
        return "".join([p.text for p in paragraphs if not p.is_boilerplate and not p.is_heading])
      else:
        return None
    except(requests.exceptions.RequestException) as e:
      print(e)
    

In [ ]:
for url in tqdm(articles):
    article = Article(url)
    print("article-->",url)
    try:
        article.download()
        article.parse()
        title = article.title 
        title_l.append(title)
        text = get_content(url)
        # print("text-->",type(text))
        if(text==None):
          continue
        text_l.append(text)

    except (newspaper.article.ArticleException) as e:
        print(e)
print(len(articles))
print(len(title_l))
print(len(text_l))

In [ ]:
newspaper_df = pd.DataFrame(
    {'Title': title_l,
    #  'URL': articles,
     'Article': text_l
    })

In [ ]:
newspaper_df['Article'].replace('', np.nan, inplace=True)

In [ ]:
newspaper_df

,Title,Article
0,JetBlue Names Paris as Second European Destina...,JetBlue plans to add Paris to its schedule sta...
1,New D.C. Metro Extension Offers Direct Dulles ...,"The new Silver Line extension on Washington, D..."
2,"Etihad, Garuda to Expand Codeshare",Etihad Airways and Garuda Indonesia have agree...
3,"ANA to Add Taipei, Shenzhen Service",All Nippon Airways on Nov. 26 will increase se...
4,"Amadeus Signs Agreements with Fareportal, Simp...",Amadeus on Tuesday announced new distribution ...
...,...,...
5658,"Banton, Boucher lead short-handed Raptors past...","“I got a lot of reps in the summer, playing fo..."
5659,Trent joins long list of injured Raptors,Trent had started all 14 games for the Raptors...
5660,Dreams come with disappointment as Canada name...,More than a few dreams officially came true in...
5661,Pacers use big fourth quarter to beat Raptors,After Toronto went ahead by 15 in the first mi...


In [ ]:
newspaper_df.dropna(subset=['Article'], inplace=True)

In [ ]:
newspaper_df

,Title,Article
0,JetBlue Names Paris as Second European Destina...,JetBlue plans to add Paris to its schedule sta...
1,New D.C. Metro Extension Offers Direct Dulles ...,"The new Silver Line extension on Washington, D..."
2,"Etihad, Garuda to Expand Codeshare",Etihad Airways and Garuda Indonesia have agree...
3,"ANA to Add Taipei, Shenzhen Service",All Nippon Airways on Nov. 26 will increase se...
4,"Amadeus Signs Agreements with Fareportal, Simp...",Amadeus on Tuesday announced new distribution ...
...,...,...
5658,"Banton, Boucher lead short-handed Raptors past...","“I got a lot of reps in the summer, playing fo..."
5659,Trent joins long list of injured Raptors,Trent had started all 14 games for the Raptors...
5660,Dreams come with disappointment as Canada name...,More than a few dreams officially came true in...
5661,Pacers use big fourth quarter to beat Raptors,After Toronto went ahead by 15 in the first mi...


In [ ]:
newspaper_df.to_csv('data.csv')
files.download('data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>